# Google Maps Jupyter Widget

This is a simple Google Maps widget that showcases two way interaction between Google Maps notebook content with a Jupyter widget.

## Configure the Javascript half of the Widget

This is the Javascript side of the widget and does not typically need modified (unless you are added features to the widget). Eventually this should be bundled as a standalone widget that can be imported from outside the notebook. For now simply execute the cell to define the widget code.

In [ ]:
%%javascript

require(['widgets/js/widget', 'widgets/js/manager', 'jquery', 'underscore'], function(widget, manager, $, _) {
    /**
     * A simple model to represent a layer on the map.
     *
     * @constructor
     */
    var Layer = Backbone.Model.extend({
        defaults: function() {
            return {
                config: {},
                type: undefined,
                visible: true
            };
        }
    });



    /**
     * A collection of layers.
     *
     * @constructor
     */
    var LayerCollection = Backbone.Collection.extend({
        model: Layer
    });



    /**
     * Override of the main widget model to intercept messages from Python
     * update Javascript state correctly.
     *
     * @constructor
     */
    var GoogleMapsModel = widget.WidgetModel.extend({
        /*
        defaults: function() {
            return {
                layers: new LayerCollection()
            };
        },
        */

        /** @override */
        initialize: function() {
            this.listenTo(
                this, 'msg:custom', _.bind(this.handleMessage, this));
            // this.set('layers', new LayerCollection());
        },

        /**
         * Handle a message from Python.
         *
         * @param {!Object} payload Payload of the message.
         */
        handleMessage: function(payload) {
            if (!this.get('layers')) {
                this.set('layers', new LayerCollection());
            }
            switch(payload.action) {
                case 'addLayer':
                    this.get('layers').add({
                        config: payload.config,
                        type: payload.type
                    });
                    break;
                case 'removeLayer':
                    console.error('removeLayer not implemented');
                    break;
            }
        }
    });

    // Register the model.
    manager.WidgetManager.register_widget_model(
        'GoogleMapsModel', GoogleMapsModel);



    /**
     * A Google Maps API widget.
     *
     * @constructor
     */
    var GoogleMapsView = widget.DOMWidgetView.extend({
        /**
         * Load the Maps API JS if needed, also prepare a deferred in case any
         * map methods are called before the map is ready.
         */
        initialize: function() {
            // Deferred to track for when the map is ready.
            this.mapsReadyDeferred = $.Deferred();

            // Dynamically adding Google Maps API JS here. Using a deferred to
            // track its load status as require returns as soon as the first
            // script loads and the Maps API triggers more scripts to append
            // which leaves a race condition where require thinks Maps API JS
            // is ready when it is not yet.
            var mapsDeferred = this.mapsDeferred = $.Deferred();
            // Another instance of this view may have already loaded the Maps
            // API JS, do not try to load it twice.
            if (window.google && window.google.maps) {
                mapsDeferred.resolve();
            } else {
                window.googleMapsCallback = function() {
                    mapsDeferred.resolve();
                };
                require(
                    ['http://maps.googleapis.com/maps/api/js?callback=googleMapsCallback'],
                    function() {},
                    function() {});
            }
        },

        /**
         * Render the map.
         */
        render: function() {
            // We must wait until the Maps API JS is ready.
            $.when(this.mapsDeferred.promise()).then(_.bind(function() {
                // Empty the views DOM. There seem to be some weird side
                // effects when you render more than one instance of this view
                // in the notebook. Cleaning the view DOM and deferring the map
                // initialization seems to work around this. It seesm almost if
                // map instances are sharing some DOM somehow.
                this.$el.empty();
                _.defer(_.bind(function() {
                    this.$map = $([
                        '<div style="height: ',
                        this.model.get('height'),
                        '; width: ',
                        this.model.get('width'),
                        ';"></div>'
                    ].join(''));
                    this.$el.append(this.$map);
                    this.map = new google.maps.Map(this.$map.get(0), {
                        center: {
                            lat: this.model.get('lat'),
                            lng: this.model.get('lng')
                        },
                        zoom: this.model.get('zoom')
                    });

                    // Notify the map when the map container changes size via
                    // the exposed properties in the model.
                    this.listenTo(
                        this.model, 'change:height', _.bind(function() {
                            this.$map.height(this.model.get('height'));
                            google.maps.event.trigger(this.map, 'resize');
                        }, this));
                    this.listenTo(
                        this.model, 'change:width', _.bind(function() {
                            this.$map.width(this.model.get('width'));
                            google.maps.event.trigger(this.map, 'resize');
                        }, this));

                    // Bind a change in the position of the map to the model.
                    google.maps.event.addListener(
                        this.map,
                        'bounds_changed',
                        _.bind(this.syncFromMap, this));

                    // Bind a change in the model (coming from the Python side)
                    // to the location of the map.
                    this.listenTo(
                        this.model,
                        'change:lat',
                        _.bind(this.syncFromModel, this));
                    this.listenTo(
                        this.model,
                        'change:lng',
                        _.bind(this.syncFromModel, this));
                    this.listenTo(
                        this.model,
                        'change:zoom',
                        _.bind(this.syncFromModel, this));

                    // Render the initial set of layers.
                    if (!this.model.get('layers')) {
                        this.model.set('layers', new LayerCollection());
                    }
                    this.model.get('layers').each(this.buildLayer, this);

                    // Bind to changes in the layers of the model to stay in
                    // sync.
                    this.listenTo(
                        this.model.get('layers'),
                        'add',
                        _.bind(this.buildLayer, this));
                    this.listenTo(
                        this.model.get('layers'),
                        'remove',
                        function() {
                            console.error('removeLayer not implemented');
                        });

                    // Even though a google.maps.Map instance should be ready
                    // immediately, it is not. This delay lets the stack clear
                    // and initial map bounds to be set.
                    _.delay(_.bind(function() {
                        this.mapsReadyDeferred.resolve();
                    }, this), 500);
                }, this));
            }, this));
        },

        /**
         * Sync the values from the map into the model.
         */
        syncFromMap: function() {
            this.model.set({
                lat: this.map.getCenter().lat(),
                lng: this.map.getCenter().lng(),
                zoom: this.map.getZoom()
            });
            // This is needed for the model to update the equivalent properties
            // on the Python instance of this view.
            this.model.save_changes();
        },

        /**
         * Move the map to match the values from the model.
         */
        syncFromModel: function() {
            this.map.setCenter(new google.maps.LatLng(
                this.model.get('lat'),
                this.model.get('lng')
            ));
            this.map.setZoom(this.model.get('zoom'));
        },

        /**
         * Add a layer to the map based on its model.
         *
         * @param {!Layer} layer The layer to add.
         */
        buildLayer: function(layer) {
            switch(layer.get('type')) {
                case 'geojsondata':
                    this.addGeoJsonLayer(layer.get('config').data);
                    break;
                case 'geojsonurl':
                    this.loadGeoJsonLayer(layer.get('config').url);
                    break;
                case 'kmlurl':
                    this.loadKmlLayer(layer.get('config').url);
                    break;
                case 'earthengine':
                    this.addEarthEngineLayer(
                        layer.get('config').mapid, layer.get('config').token);
                    break;
            }
        },

        /**
         * Adds GeoJSON to the map.
         *
         * @param {!Object} data A GeoJSON object.
         */
        addGeoJsonLayer: function(data) {
            // Defer until map is ready.
            this.mapsReadyDeferred.done(_.bind(function() {
                this.map.data.addGeoJson(data);
            }, this));
        },

        /**
         * Adds a URL location of GeoJSON to the map.
         *
         * @param {string} url The URL of the GeoJSON file to load.
         */
        loadGeoJsonLayer: function(url) {
            // Defer until map is ready.
            this.mapsReadyDeferred.done(_.bind(function() {
                this.map.data.loadGeoJson(url);
            }, this));
        },

        /**
         * Add a KML layer to the map.
         *
         * @param {string} url The URL of the KML file to load.
         */
        loadKmlLayer: function(url) {
            // Defer until map is ready.
            this.mapsReadyDeferred.done(_.bind(function() {
                new google.maps.KmlLayer({
                    url: url,
                    map: this.map
                });
            }, this));
        },

        /**
         * Add an Earth Engine layer to the map.
         *
         * @param {string} mapid The id of the Earth Engine layer.
         * @param {string} token The OAuth token to authenticate with.
         */
        addEarthEngineLayer: function(mapid, token) {
            // Defer until map is ready.
            this.mapsReadyDeferred.done(_.bind(function() {
                var eeMapOptions = {
                    getTileUrl: function(tile, zoom) {
                        var url = [
                            'https://earthengine.googleapis.com/map',
                            mapid,
                            zoom,
                            tile.x,
                            tile.y
                        ].join('/');
                        url += '?token=' + token;
                        return url;
                    },
                    tileSize: new window.google.maps.Size(256, 256),
                    opacity: 1.0,
                };

                // Create the overlay map type.
                var mapType = new window.google.maps.ImageMapType(eeMapOptions);

                // Overlay the Earth Engine generated layer.
                this.map.overlayMapTypes.push(mapType);
            }, this));
        }
    });

    // Register the view.
    manager.WidgetManager.register_widget_view(
        'GoogleMapsView', GoogleMapsView);
});


## Configure the Python half of the Widget

This is the Python side of the widget and does not typically need modified (unless you are added features to the widget). Eventually this should be bundled as a standalone widget that can be imported from outside the notebook. For now simply execute the cell to register the widget for use.

In [ ]:
from ipywidgets import widgets
import traitlets

class GoogleMapsView(widgets.DOMWidget):
    """Google Maps API widget."""
    _model_name = traitlets.Unicode('GoogleMapsModel', sync=True)
    _view_name = traitlets.Unicode('GoogleMapsView', sync=True)
    lat = traitlets.CFloat(0, sync=True)
    lng = traitlets.CFloat(0, sync=True)
    zoom = traitlets.CInt(2, sync=True)

    height = traitlets.CUnicode('300px', sync=True)
    width = traitlets.CUnicode('400px', sync=True)

    layers = traitlets.List([], sync=False)

    def addGeoJsonLayer(self, data):
        """Adds a dictionary of GeoJSON to the map.

        NOTE: It is likely if you are using a third party GeoJSON library you
        will have to first serialize the data into a simple dictionary before
        passing the data to this method.

        Args:
            data: A simple python dictionary of GeoJSON data.
        """
        self.send({
            'action': 'addLayer',
            'type': 'geojsondata',
            'config': {'data': data}
        })

    def loadGeoJsonLayer(self, url):
        """Adds a URL location of GeoJSON to the map.

        Args:
            url: The URL of the GeoJSON file.
        """
        self.send({
            'action': 'addLayer',
            'type': 'geojsonurl',
            'config': {'url': url}
        })

    def loadKmlLayer(self, url):
        """Adds a KML layer to the map.

        Args:
            url: The URL of the KML file.
        """
        self.send({
            'action': 'addLayer',
            'type': 'kmlurl',
            'config': {'url': url}
        })

    def addEarthEngineLayer(self, image, vis_params):
        """Adds an Earth Engine layer to the map.

        Args:
            image: The ee.Image to display.
            vis_params: Dictionary of visualization parameters.
        """
        mapid = image.getMapId(vis_params)
        self.send({
            'action': 'addLayer',
            'type': 'earthengine',
            'config': {
                'mapid': mapid['mapid'],
                'token': mapid['token']
            }
        })


## Import Dependencies

Some basic dependencies in order to display results of the examples below.

In [ ]:
from IPython.display import display

## Define and display a Map Widget

This example code will create and display a simple Google Map.

In [ ]:
map1 = GoogleMapsView(lng=-119.2, lat=36.3, zoom=4, height='240px', width='800px')
display(map1)

## Create a second Map Widget and link up panning

This example shows how you can link Google Maps widgets together for a linked multi display for cases when you want to showcase multiple datasources together. Note, as you pan and move around the map, the second map will stay in sync with the map above.

In [ ]:
map2 = GoogleMapsView(height='240px', width='800px')
widgets.jslink((map1, 'lat'), (map2, 'lat'))
widgets.jslink((map1, 'lng'), (map2, 'lng'))
widgets.jslink((map1, 'zoom'), (map2, 'zoom'))
display(map2)

## Display GeoJSON

This example shows two ways of loading GeoJSON data. You can load data directly in Python by creating simple python dictionaries of GeoJSON or point to an external URL.

In [ ]:
geojson_map = GoogleMapsView(lng=125.6, lat=10.1, zoom= 10, height='240px', width='800px')
# This loads GeoJSON directly from a Python dictionary.
geojson_map.addGeoJsonLayer({
    'type': 'Feature',
    'geometry': {
        'type': 'Point',
        'coordinates': [125.6, 10.1]
    },
    'properties': {
        'name': 'Dinagat Islands'
    }
})
display(geojson_map)

In [ ]:
# This loads a URL of GeoJSON (to the same map above).
geojson_map.loadGeoJsonLayer('https://storage.googleapis.com/maps-devrel/google.json')
# Move the map to see the data.
geojson_map.lng = 137
geojson_map.lat = -28
geojson_map.zoom = 4

## Display a KML layer

This example shows how to load a basic KML layer on a map.

In [ ]:
# Create a simple map.
kml_map = GoogleMapsView(lng=-87.74947400000002, lat=41.89757302024969, zoom=8, height='240px', width='800px')
# Add the layer.
kml_map.loadKmlLayer('http://googlemaps.github.io/js-v2-samples/ggeoxml/cta.kml')
display(kml_map)

## Display a Google Earth Engine layer

This example shows a basic image from Google Earth Engine on the map (above).

WARNING: For these examples to work you must have your Jupyter notebook instance properly authenticated. See the Google Earth Engine guides on how to do this if you are interested.

In [ ]:
import ee

In [ ]:
ee_image_map = GoogleMapsView(lng=-119.2, lat=36.3, zoom=4, height='240px', width='800px')
dem = ee.Image('USGS/SRTMGL1_003')
ee_image_map.addEarthEngineLayer(image=dem, vis_params={'min': 0, 'max': 3000})
display(ee_image_map)

## Define some Earth Engine image layers and visualization parameters

This example shows data from three different sources with some visualization parameters set.

In [ ]:
# Landsat 8
l8 = ee.ImageCollection('LANDSAT/LC8_L1T_TOA').filterDate('2015-09-01', '2015-11-01').median()

# Skybox
skybox1 = ee.ImageCollection('S4G_BETA/MULTISPECTRAL').mosaic();

# Calculated NDVI
skybox_ndvi = skybox1.normalizedDifference(['N','R'])

# Create map widgets.
map3a = GoogleMapsView(lng=-57.8106689453125, lat=2.6474491435545797, zoom=6, height='240px', width='800px')
map3b = GoogleMapsView(height=map3a.height, width=map3a.width)
map3c = GoogleMapsView(height=map3a.height, width=map3a.width)

# Link the map widgets.
widgets.jslink((map3a, 'lat'), (map3b, 'lat'))
widgets.jslink((map3a, 'lng'), (map3b, 'lng'))
widgets.jslink((map3a, 'zoom'), (map3b, 'zoom'))
widgets.jslink((map3a, 'lat'), (map3c, 'lat'))
widgets.jslink((map3a, 'lng'), (map3c, 'lng'))
widgets.jslink((map3a, 'zoom'), (map3c, 'zoom'))

# Add the layers to the maps.
map3a.addEarthEngineLayer(image=l8, vis_params={'min': 0, 'max': 0.3, 'bands': 'B4,B3,B2'})
map3b.addEarthEngineLayer(image=skybox1, vis_params={'min': 0, 'max': 200, 'bands': 'R,G,B'})
map3c.addEarthEngineLayer(image=skybox_ndvi, vis_params={'min': 0, 'max': 1})

display(map3a)
display(map3b)
display(map3c)